In [1]:
#Notebook encargada de crear el dataset Wind

In [1]:
%config Completer.use_jedi = False

In [2]:
import os
import pandas as pd
from pathlib import Path
import georasters as gr
from osgeo import gdal
import datetime
import numpy as np
from pathlib import Path
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
from osgeo import gdal # Import the GDAL library

In [4]:
lista_paths = list()
base = '/opt/datos/gfs/wind_00/gfs.0p25.'
ext = '.grib2'
start_date = "2015-01-15 00:00"
finish_date = "2021-08-21 00:00"
idx = pd.date_range(start_date, finish_date, freq='24H').strftime("%Y%m%d%H")
f = ['.f003', '.f006', '.f009', '.f012', '.f015', '.f018', '.f021', '.f024']

for time in idx:
    for k in f:
        time_new = base+time+k+ext
        lista_paths.append(time_new)

In [5]:
print(lista_paths[1500])
print(lista_paths.index('/opt/datos/gfs/wind_00/gfs.0p25.2015072100.f015.grib2'))

/opt/datos/gfs/wind_00/gfs.0p25.2015072100.f015.grib2
1500


In [12]:
print(grib_t.GetRasterBand(1).ReadAsArray().shape)

(137, 157)


In [16]:
X = np.zeros((19288,2,137,157))
for i,path in enumerate(lista_paths): #i va de 0 a 19287
    grib = gdal.Open(path)
    
    #band1 = grib.GetRasterBand(1) #U component wind band: 700mb
    #band2 = grib.GetRasterBand(2) #V component wind band: 700mb
    
    band1 = grib.GetRasterBand(3) #U component wind band: 850mb
    band2 = grib.GetRasterBand(4) #V component wind band: 850mb
    
    data1 = band1.ReadAsArray()
    data2 = band2.ReadAsArray()

    X[i,0,:,:] = data1
    X[i,1,:,:] = data2

In [17]:
X.shape

(19288, 2, 137, 157)

In [15]:
np.save("/opt/datos/dataset/gfs/wind/X_700.npy", X)

In [ ]:
np.save("/opt/datos/dataset/gfs/wind/X_850.npy", X)

### Armado Dataset: (u_wind, v_wind, lluvia) 

In [3]:
rain = np.load('/opt/datos/dataset/gfs/Y.npy')
#wind_700 = np.load('/opt/datos/dataset/gfs/wind/X_700.npy')
wind_850 = np.load('/opt/datos/dataset/gfs/wind/X_850.npy')

In [4]:
print(rain.shape)

(19288, 137, 157)


In [4]:
#X_700 = np.zeros((19288,3,137,157))
X_850 = np.zeros((19288,3,137,157))

for i in range(rain.shape[0]):
    #X_700[i,0:3,:,:] = rain[i], wind_700[i,0,:,:], wind_700[i,1,:,:] #combinar rain con u y v wind 700
    X_850[i,0:3,:,:] = rain[i], wind_850[i,0,:,:], wind_850[i,1,:,:] #combinar rain con u y v wind 850
    if(i==10000):
        print("50 %")

50 %


In [13]:
X_700[19287,1,:,:]

array([[ 2.24000001,  2.19000006,  2.11999989, ..., -0.44      ,
        -0.29999998, -0.19999999],
       [ 2.5999999 ,  2.57999992,  2.53999996, ..., -0.14999999,
         0.        ,  0.22999999],
       [ 3.01999998,  2.98000002,  2.95000005, ...,  0.34      ,
         0.44999999,  0.69999999],
       ...,
       [ 2.56999993,  2.71000004,  3.02999997, ...,  5.82999992,
         5.75      ,  5.80999994],
       [ 3.13999987,  3.02999997,  3.0999999 , ...,  5.36999989,
         5.4000001 ,  5.55999994],
       [ 3.49000001,  3.61999989,  3.5999999 , ...,  5.07999992,
         5.25999975,  5.42999983]])

In [ ]:
np.save("/opt/datos/dataset/gfs/wind/X_rain700.npy", X_700)

In [5]:
np.save("/opt/datos/dataset/gfs/wind/X_rain850.npy", X_850)